In [1]:
%%capture --no-stderr
%pip install -U langsmith ragas numpy openai

In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [5]:
import langsmith

client = langsmith.Client()
dataset_url = (
    "https://smith.langchain.com/public/56fe54cd-b7d7-4d3b-aaa0-88d7a2d30931/d"
)
dataset_name = "BaseCamp Q&A"
client.clone_public_dataset(dataset_url)

Dataset(name='BaseCamp Q&A', description='Taken from: https://basecamp.com/handbook', data_type=<DataType.kv: 'kv'>, id=UUID('6f330976-9a78-4fd3-9718-5ebba585af98'), created_at=datetime.datetime(2025, 3, 28, 19, 29, 35, 17929, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 3, 28, 19, 29, 35, 17929, tzinfo=datetime.timezone.utc), example_count=0, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None)

In [6]:
import io
import os
import zipfile

import requests


url = "https://storage.googleapis.com/benchmarks-artifacts/basecamp-data/basecamp-data.zip"

response = requests.get(url)


with io.BytesIO(response.content) as zipped_file:
    with zipfile.ZipFile(zipped_file, "r") as zip_ref:
        zip_ref.extractall()

data_dir = os.path.join(os.getcwd(), "data")
docs = []
for filename in os.listdir(data_dir):
    if filename.endswith(".md"):
        with open(os.path.join(data_dir, filename), "r") as file:
            docs.append({"file": filename, "content": file.read()})

In [7]:
from typing import List

import numpy as np
import openai
from langsmith import traceable


class VectorStoreRetriever:
    def __init__(self, docs: list, vectors: list, oai_client):
        self._arr = np.array(vectors)
        self._docs = docs
        self._client = oai_client

    @classmethod
    async def from_docs(cls, docs, oai_client):
        embeddings = await oai_client.embeddings.create(
            model="text-embedding-3-small", input=[doc["content"] for doc in docs]
        )
        vectors = [emb.embedding for emb in embeddings.data]
        return cls(docs, vectors, oai_client)

    @traceable
    async def query(self, query: str, k: int = 5) -> List[dict]:
        embed = await self._client.embeddings.create(
            model="text-embedding-3-small", input=[query]
        )

        scores = np.array(embed.data[0].embedding) @ self._arr.T
        top_k_idx = np.argpartition(scores, -k)[-k:]
        top_k_idx_sorted = top_k_idx[np.argsort(-scores[top_k_idx])]
        return [
            {**self._docs[idx], "similarity": scores[idx]} for idx in top_k_idx_sorted
        ]

In [8]:
from langsmith import traceable
from langsmith.wrappers import wrap_openai


class NaiveRagBot:
    def __init__(self, retriever, model: str = "gpt-4o-mini"):
        self._retriever = retriever

        self._client = wrap_openai(openai.AsyncClient())
        self._model = model

    @traceable
    async def get_answer(self, question: str):
        similar = await self._retriever.query(question)
        response = await self._client.chat.completions.create(
            model=self._model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful AI assistant."
                    " Use the following docs to help answer the user's question.\n\n"
                    f"## Docs\n\n{similar}",
                },
                {"role": "user", "content": question},
            ],
        )


        return {
            "answer": response.choices[0].message.content,
            "contexts": [str(doc) for doc in similar],
        }

In [9]:
retriever = await VectorStoreRetriever.from_docs(docs, openai.AsyncClient())
rag_bot = NaiveRagBot(retriever)

In [10]:
response = await rag_bot.get_answer("How much time off do we get?")
response["answer"][:150]

'At 37signals, employees are entitled to 18 days of paid time off (PTO) each year, in addition to 11 local holidays. This vacation time is prorated bas'

In [11]:
from langchain.smith import RunEvalConfig
from ragas.integrations.langchain import EvaluatorChain
from ragas.metrics import (
    answer_correctness,
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness,
)


evaluators = [
    EvaluatorChain(metric)
    for metric in [
        answer_correctness,
        answer_relevancy,
        context_precision,
        context_recall,
        faithfulness,
    ]
]
eval_config = RunEvalConfig(custom_evaluators=evaluators)

/workspaces/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
results = await client.arun_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=rag_bot.get_answer,
    evaluation=eval_config,
)

View the evaluation results for project 'pertinent-passenger-58' at:
https://smith.langchain.com/o/163bb599-804b-58d8-a6e7-07b73cf89382/datasets/6f330976-9a78-4fd3-9718-5ebba585af98/compare?selectedSessions=3aee663d-5c39-4475-bcff-e49d831105e4

View all tests for Dataset BaseCamp Q&A at:
https://smith.langchain.com/o/163bb599-804b-58d8-a6e7-07b73cf89382/datasets/6f330976-9a78-4fd3-9718-5ebba585af98
[-------------------------------------------->     ] 19/21

Error evaluating run 878f8b72-bc1f-4585-bc94-aea43c90e422 with EvaluatorChain: APIConnectionError('Connection error.')
Traceback (most recent call last):
  File "/workspaces/RAG/.venv/lib/python3.12/site-packages/openai/_base_client.py", line 1500, in _request
    response = await self._client.send(
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/RAG/.venv/lib/python3.12/site-packages/httpx/_client.py", line 1629, in send
    response = await self._send_handling_auth(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/RAG/.venv/lib/python3.12/site-packages/httpx/_client.py", line 1657, in _send_handling_auth
    response = await self._send_handling_redirects(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/RAG/.venv/lib/python3.12/site-packages/httpx/_client.py", line 1694, in _send_handling_redirects
    response = await self._send_single_request(request)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/RAG

[------------------------------------------------->] 21/21